In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import yaml
import tweepy
import numpy as npxs
import timeit


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
consumer_key = 'Q2bQGR5l4rm8V9wLdPAWVlygx'
consumer_secret = '0gIbTawUIQzsQqq2Tw1cPjtosorYx8dHoNK0gyqFGng7xNovsJ'
access_token = '1507957215537283072-b0EZy0V42NKpTuJKOvyVm1sswGwFb2'
access_token_secret = 'lLieNQUS4copiahln6ifoZAuMYux19J08GYDUvjli3bxQ'

In [3]:
def auth():
    try:
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        print('It worked')
      
        
    except:
        print('An error occurred during the authentication')
    return api

In [4]:
# function to remove URL
def remove_url(txt):
    return ' '.join(re.sub('([^0-9A-Za-z \t])|(\w+:\/\/\S+)', '', txt).split())

# function to remove time
def remove_time(datetime):
    return str(datetime).split(' ')[0]

In [5]:
def search_by_hashtag(api, date_until, words, max):
    df = pd.DataFrame(columns=['id', 'created_at', 'username', 'location', 'following', 
                               'followers', 'retweetcount', 'text']) 
    tweets = tweepy.Cursor(api.search_tweets, q=words, lang='en', until=date_until, tweet_mode='extended').items(max) 
    list_tweets = [tweet for tweet in tweets] 
    
    for tweet in list_tweets: 
        id = tweet.id
        created_at = remove_time(tweet.created_at)
        username = tweet.user.screen_name 
        location = tweet.user.location 
        following = tweet.user.friends_count 
        followers = tweet.user.followers_count  
        retweetcount = tweet.retweet_count 
    
        try: 
            text = tweet.retweeted_status.full_text 
        except AttributeError: 
            text = tweet.full_text 
        text = remove_url(text)

        tweets = [id, created_at, username, location, following, followers, retweetcount, text]

        df.loc[len(df)] = tweets # add current tweet to the last
          
    return df

In [6]:
api = auth()
words = 'Russia -filter:retweets'
date_until = '2022-04-08'
df = search_by_hashtag(api, date_until, words, 1000)
df.head()

It worked


,id,created_at,username,location,following,followers,retweetcount,text
0,1512218663151190021,2022-04-07,ricardus_13,,96,83,0,olliecarroll One reason people insist Russia d...
1,1512218655530131458,2022-04-07,ProudDem6,,1978,1369,0,JudiciaryGOP We still cant believe republicans...
2,1512218655345369088,2022-04-07,stinkymouth,"Las Vegas, NV",2237,194,0,hankmoody2020 thehill USA constantly looking f...
3,1512218654137536514,2022-04-07,WLockefeer,,30,43,0,addexteram HollyGo55 KyivIndependent Letting R...
4,1512218650694103065,2022-04-07,sethkushondeck,Basedworld,471,407,0,SwansonElise R66D2 aaronjmate DrGee777 We are ...


In [7]:
#df = pd.read_csv('processed_tweets')
#df.head(100)